## Measuring the full embryo length

The full embryo length is estimated from the major axis of the ellipse that has the same second moment as the binary image that represents the embryo.
With the exception of the VNC, the pixel values inside the embryo area are low and similar to the background pixels.
To split them, we first equalize the image histogram, followed by a low threshold method (triangle method).
After we have the binary image, we can use `skimage.measure.regionprops` to measure the embryo length.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from tifffile import imread
from skimage.exposure import equalize_hist

from pasnascope import full_embryo_length, utils

experiment_name = '20240515'
root_dir = Path.cwd().parent
project_dir = root_dir.joinpath('data', experiment_name)

#### Visualizing the method

Compare the generated measurement with the original image.

In [ ]:
embs = sorted(project_dir.joinpath('embs').glob('*ch2.tif'), key=utils.emb_number)
i = 5

print(embs[i].stem)

img = imread(embs[i], key=range(0,100))
img = np.average(img, axis=0)
equalized = equalize_hist(img) 
bin_img = full_embryo_length.binarize(equalized)
full_embryo_length.view_full_embryo_length(bin_img, img)

#### Measuring the embryo length

Displays the embryo length for all embryos found in the current project.

In [ ]:
embs = sorted(project_dir.joinpath('embs').glob('*ch2.tif'), key=utils.emb_number)

for emb in embs:
    print(emb.stem, full_embryo_length.measure(emb))

Compares embryo size with annotated data.

In [ ]:
csv_path = project_dir.joinpath('emb_sizes', 'embryo_size.csv')
try:
    annotated_data = full_embryo_length.get_annotated_data(csv_path)

    # maps annotated_id: embryo_id
    name_LUT = {1:1,2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9, 10:10, 11:11, 12:12, 13:13, 14:14, 15:15, 16:16, 17:17, 18:18, 19:19, 20:20, 23:22, 24:23}

    embs = sorted(project_dir.joinpath('embs').glob('*ch2.tif'), key=utils.emb_number)

    measured = {utils.emb_number(e.stem): full_embryo_length.measure(e) for e in embs}

    annotated = {}
    for ann_id, size in annotated_data:
        emb_id = name_LUT.get(ann_id, None)
        if emb_id is None:
            continue
        annotated[emb_id] = size
        
    y = [annotated[k] for k in sorted(annotated.keys())]
    z = [measured[k] for k in sorted(measured.keys())]

    x = np.arange(len(measured))
    fig, ax = plt.subplots()
    ax.plot(x, y, label='annotated')
    ax.plot(x, z, label='measured')
    ax.legend()
    fig.suptitle(f'Embryo size measurement - {experiment_name}')
except FileNotFoundError:
    print('Could not find the file with annotated data. Path received:')
    print(csv_path)